# Create a single node vector database 

TODO:
- ~~pandas df --> lance DB~~
- ~~spark df --> pyarrow --> lance DB~~
- ~~batch load in arrow ds and write to lance db~~
- [ ] figure out "hyperparam" tuning of sub vectors + partitions for IVF index
- [ ] Lance x Ray https://lancedb.github.io/lance/integrations/ray.html
  - Consider using Ray to write to Lance? 

Goal is 10M QPM @ $150k (i.e. $15/thousand queries)

Current is 24 DBU/H for 5 QPM .4 DBU/M

10M/5 = 2M nodes running in parallel to hit 10M QPM

2M * 24DBU/H * 1/60 = 800k DBU * .55/DBU = 440k 

In [0]:
%pip install lancedb numpy  
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("catalog", "jon_cheung", "catalog")
catalog = dbutils.widgets.get("catalog")

dbutils.widgets.text("schema", "vizio_poc", "schema")
schema = dbutils.widgets.get("schema")

dbutils.widgets.text("lance_table_name", "vizio_poc", "lance_table_name")
lance_table_name = dbutils.widgets.get("lance_table_name")

dbutils.widgets.text("num_vectors", "1_000_000_000", "num_vectors")
num_vectors = int(dbutils.widgets.get("num_vectors"))

dbutils.widgets.text("num_test_vectors", "100", "num_test_vectors")
num_test_vectors = int(dbutils.widgets.get("num_test_vectors"))

In [0]:
## Parameters
# num_vectors = 1_000_000_000 ## number of vectors to build LanceDB from
# catalog = 'jon_cheung'
# schema = 'vizio_poc'
# n_test_vectors = 100 


lance_db_uri = "/tmp/lancedb"
audio_parquet_path = f'/Volumes/{catalog}/{schema}/{lance_table_name}'
# https://lancedb.github.io/lancedb/ann_indexes/#how-to-choose-num_partitions-and-num_sub_vectors-for-ivf_pq-index
num_partitions = 25
# rows_per_partition = 500_000 ## IVF index parameter; While a very high num_partitions makes individual partition searches faster, there's a point of diminishing returns where the overhead of managing too many small partitions or having to search more partitions (via nprobes) can negate the benefit. However, compared to a low number of partitions (which would lead to large, slow-to-scan partitions), a higher num_partitions is generally better for maximizing throughput.
num_sub_vectors = 5 ## IVF index parameter; The number should be a factor of the vector dimension. Because PQ is a lossy compression of the original vector, a higher num_sub_vectors usually results in less space distortion, and thus yields better accuracy.

print(audio_parquet_path)

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS ${catalog}.${schema}.${lance_table_name}

## 1. Preprocess Spark Dataframe for Lance DB
#### Spark dataframe --> Parquet --> PyArrow


In [0]:
# Write samples to parquet if the directory is empty so we can write it to LanceDB via PyArrow
import os

if not os.path.exists(audio_parquet_path) or not os.listdir(audio_parquet_path):
    sdf = spark.read.table('jonathan_mcfadden.vizio_poc.audio_test').limit(num_vectors).select("id", "list_col")
    sdf.write.mode('overwrite').parquet(audio_parquet_path)

In [0]:
# Load in Parquet as PyArrow Dataset and modify schema for LanceDB
import pyarrow as pa
import pyarrow.dataset as ds

# Define a custom PyArrow schema
pyarrow_schema = pa.schema(
    [
        pa.field("id", pa.int64()),
        pa.field("list_col", pa.list_(pa.float16(), 35)),   # Fixed size list
    ]
)

In [0]:
def get_batches_from_parquet(parquet_path: str, schema: pa.Schema, batch_size: int = 1024):
    """
    Reads a Parquet file in chunks and yields PyArrow RecordBatches.
    """
    dataset = ds.dataset(parquet_path, format="parquet", schema=schema)
    scanner = dataset.scanner(batch_size=batch_size) # Specify batch_size for iteration
    for batch in scanner.to_batches():
        yield batch

In [0]:
from tqdm import tqdm # Use tqdm.auto for intelligent display (console/notebook)

def get_batches_from_parquet_with_progress(parquet_path: str, schema: pa.Schema, batch_size: int = 4096):
    """
    Reads a Parquet file in chunks and yields PyArrow RecordBatches,
    displaying progress using tqdm.
    """
    dataset = ds.dataset(parquet_path, format="parquet", schema=schema)

    # Estimate total number of rows for tqdm.
    # Note: dataset.count_rows() can be slow for very large datasets if metadata isn't optimized.
    # If performance is an issue here, you might need to pre-calculate or use an estimate.
    try:
        total_rows = dataset.count_rows()
        total_batches = np.ceil(total_rows / batch_size)
    except Exception:
        # Fallback if count_rows fails or is too slow.
        # tqdm will then run without a fixed total, just showing counts.
        total_rows = None
        total_batches = None
        print("Warning: Could not determine total rows for precise tqdm progress. Progress will be based on batches.")

    scanner = dataset.scanner(batch_size=batch_size)

    # Wrap the scanner.to_batches() with tqdm
    # We use `total_batches` for tqdm's 'total' argument.
    with tqdm(total=total_batches, unit="batch", desc="Ingesting Parquet Batches") as pbar:
        for batch in scanner.to_batches():
            yield batch
            pbar.update(1) # Manually update progress for each yielded batch
            pbar.set_postfix({"rows_in_batch": len(batch)})

## 2. Create LanceDB and index


In [0]:
%sh
mkdir -p /tmp/lancedb

In [0]:
import numpy as np
import lancedb


db = lancedb.connect(lance_db_uri)

if lance_table_name in db.table_names():
    db.drop_table(lance_table_name)
    print(f"Dropped existing table: {lance_table_name}")

In [0]:
table_arrow = db.create_table(
    lance_table_name,
    data=get_batches_from_parquet_with_progress(audio_parquet_path, pyarrow_schema, batch_size=200_000),
    mode="overwrite"
)


table_arrow.create_index(
        metric="l2",
        vector_column_name="list_col",
        num_partitions=20,
        num_sub_vectors=num_sub_vectors
    )



In [0]:
# LanceDB Cloud runs distributed indexing with Ray

## 3. Test inference 
3.1 Single-thread inference  
3.2 Thread pool inference

In [0]:
import time
import os
import pandas as pd

os.environ["LANCE_CPU_THREADS"] = "96"
os.environ["LANCE_IO_THREADS"] = "96"

def create_arrays(n, dimensions):
    return [np.random.randint(0, 256, size=dimensions).astype(np.float16) for _ in range(n)]

def search_database(table, vector, limit):
    """
    Helper function to perform a single LanceDB search query.
    This function will be executed by each thread in the thread pool.
    """
    local_db = lancedb.connect(lance_db_uri) # Connect in each process
    table = local_db.open_table(lance_table_name)

    return table.search(vector).limit(limit).to_pandas()

test_micro_batch = create_arrays(n_test_vectors, dimensions=35)

start = time.time()
search_database(lance_table_name, 
                test_micro_batch, 
                limit=1)
run_time = time.time() - start
print(f"Batch search completed in {run_time:.2f} seconds.")

In [0]:
# import pandas as pd
# import numpy as np
# import random

# num_vectors = 1_000_000_000
  
# def create_arrays(n, dimensions):
#     return [np.random.randint(0, 256, size=dimensions).astype(np.float16) for _ in range(n)]
  

# data = pd.DataFrame(
#     {
#         "vector": create_arrays(num_vectors, dimensions=35),
#         "id": np.arange(num_vectors),
#     }
# )
# tbl = db.create_table("my_table_pandas_100m", data, mode="overwrite")
# # Create IVF index on top of table to improve latency but decrease accuracy
# # https://lancedb.github.io/lancedb/ann_indexes/#how-to-choose-num_partitions-and-num_sub_vectors-for-ivf_pq-index
# # - num_sub_vectors = The number should be a factor of the vector dimension. Because PQ is a lossy compression of the original vector, a higher num_sub_vectors usually results in less space distortion, and thus yields better accuracy.
# # - num_partitions = While a very high num_partitions makes individual partition searches faster, there's a point of diminishing returns where the overhead of managing too many small partitions or having to search more partitions (via nprobes) can negate the benefit. However, compared to a low number of partitions (which would lead to large, slow-to-scan partitions), a higher num_partitions is generally better for maximizing throughput.
# tbl.create_index(metric="l2", num_partitions=1000, num_sub_vectors=5)